In [1]:
(require booleans redex/reduction-semantics)
;; seems to speed things up a little
(set-cache-size! 200)

Some possibly helpful laws:

- Posulate 1: `¬( GO ∨ (RES ∧ SEL))` implies all `E'` is `0` and all `kn` are `0`.
- - Correlary 1: This holds for `¬(GO ∨ SEL)`
- - Correlary 2: This holds for `¬(GO ∨ RES)`
- Postulate 2: forall `n` `m`, `m != n`, `¬ (kn ^ km)`, that is at most one `k` wire is selected.


In [2]:
(define/ppl start #:no-check
  (
   ;; these come from then othing
   (l0 = GO)
   (lsel = false)
   ;; SEL
   (SEL = (or lsel rsel))
   ;; the synchonizer
   (K0 = (and left (and right both)))
   (left = (or l0 lem))
   (right = (or r0 rem))
   (lem = (and SEL (and RES (not lsel))))
   (rem = (and SEL (and RES (not rsel))))
   (both = (or l0 r0))))

(l0 = GO)
(lsel = false)
(SEL = (or lsel rsel))
(K0 = (and left (and right both)))
(left = (or l0 lem))
(right = (or r0 rem))
(lem = (and SEL (and RES (not lsel))))
(rem = (and SEL (and RES (not rsel))))
(both = (or l0 r0))


In [3]:
(assert-same (term start) (term ((K0 = r0) (SEL = rsel) (K1 = r1))))

equal-class: the diff:
 '((("a broken clause:" (K1 = r1))
   ((GO = true) (rsel = true) (r0 = true) (RES = true))
   ((l0 = true)
    (lsel = false)
    (SEL = true)
    (K0 = true)
    (left = true)
    (right = true)
    (lem = true)
    (rem = false)
    (both = true))
   ((K0 = true) (SEL = true) (K1 = r1)))
  (("a broken clause:" (K1 = r1))
   ((GO = true) (rsel = true) (r0 = true) (RES = false))
   ((l0 = true)
    (lsel = false)
    (SEL = true)
    (K0 = true)
    (left = true)
    (right = true)
    (lem = false)
    (rem = false)
    (both = true))
   ((K0 = true) (SEL = true) (K1 = r1)))
  (("a broken clause:" (K1 = r1))
   ((GO = true) (rsel = true) (r0 = true) (RES = ⊥))
   ((l0 = true)
    (lsel = false)
    (SEL = true)
    (K0 = true)
    (left = true)
    (right = true)
    (lem = ⊥)
    (rem = false)
    (both = true))
   ((K0 = true) (SEL = true) (K1 = r1)))
  (("a broken clause:" (K1 = r1))
   ((GO = true) (rsel = true) (r0 = false) (RES = true))
   ((l0 = true)
   

   ((K0 = ⊥) (SEL = ⊥) (K1 = r1))))
  context...:
   eval-one-top12
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:510:0: call-with-custodian-shutdown
   /Applications/Racket/racket/collects/racket/private/more-scheme.rkt:148:2: call-with-break-parameterization
   .../more-scheme.rkt:261:28
   /Applications/Racket/racket/share/pkgs/sandbox-lib/racket/sandbox.rkt:878:5: loop


In [4]:
(define/ppl translated #:no-check
  (convert-P start))

((+ l0) = (+ GO))
((- l0) = (- GO))
((+ lsel) = false)
((- lsel) = true)
((+ SEL) = (or (+ lsel) (+ rsel)))
((- SEL) = (and (- lsel) (- rsel)))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ l0) (+ lem)))
((- left) = (and (- l0) (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (and (+ RES) (- lsel))))
((- lem) = (or (- SEL) (or (- RES) (+ lsel))))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or (+ l0) (+ r0)))
((- both) = (and (- l0) (- r0)))


Step 1: By rules of `and` and `or`

In [5]:
(define/ppl step0 #:no-check
 (propigate/remove* translated
    (+ l0) (- l0) (+ lsel) (- lsel)))

((+ SEL) = (or false (+ rsel)))
((- SEL) = (and true (- rsel)))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ GO) (+ lem)))
((- left) = (and (- GO) (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (and (+ RES) true)))
((- lem) = (or (- SEL) (or (- RES) false)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or (+ GO) (+ r0)))
((- both) = (and (- GO) (- r0)))


In [6]:
(define/ppl step1
  replace* step0 
  [(or false (+ rsel)) (+ rsel)]
  [(and true (- rsel)) (- rsel)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ GO) (+ lem)))
((- left) = (and (- GO) (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (and (+ RES) true)))
((- lem) = (or (- SEL) (or (- RES) false)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or (+ GO) (+ r0)))
((- both) = (and (- GO) (- r0)))


In [7]:
(define/ppl step-before-case
  replace*
  step1
  [(and (+ RES) true) (+ RES)]
  [(or (- RES) false) (- RES)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or (+ GO) (+ lem)))
((- left) = (and (- GO) (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or (+ GO) (+ r0)))
((- both) = (and (- GO) (- r0)))


By +x and -x = 0 and law of the excluded middle we can have
three cases:

1. `+GO` = `-GO` = 0
2. `+GO` = 1, `-GO` = 0
3. `+GO` = 0, `-GO` = 1

# CASE 1 `+GO` = `-GO` = 0 (`GO` is unknown)

In [8]:
(define/ppl case1-step1 #:no-check
  (replace* 
   step-before-case
   [(+ GO) false]
   [(- GO) false]))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or false (+ lem)))
((- left) = (and false (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or false (+ r0)))
((- both) = (and false (- r0)))


by the laws of and/or on false

In [9]:
(define/ppl case1-step2
  replace* case1-step1
           [(or false (+ r0)) (+ r0)]
           [(and false (- r0)) false]
           [(or false (+ lem)) (+ lem)]
           [(and false (- lem)) false]
           [(or false (and (+ SEL) (and (+ RES) (- lsel))))
            (and (+ SEL) (and (+ RES) (- lsel)))]
           [(and false (or (- SEL) (or (- RES) (+ lsel))))
            false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (+ lem))
((- left) = false)
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (+ r0))
((- both) = false)


In [10]:
(define/ppl case1-step3
   propigate/remove* case1-step2
                     (+ both) (- both)
                     (+ left) (- left))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (+ right) (+ r0))))
((- K0) = (or false (or (- right) false)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [11]:
(define/ppl case1-step4
   replace* case1-step3
            [(or false (or (- right) false))
             (- right)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (+ right) (+ r0))))
((- K0) = (- right))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [12]:
(define/ppl case1-step5
  propigate/remove*
  case1-step4
  (+ right) (- right))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (or (+ r0) (+ rem)) (+ r0))))
((- K0) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [13]:
(define/ppl case1-step6
  propigate*
  case1-step5
  (- SEL) (+ SEL))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (or (+ r0) (+ rem)) (+ r0))))
((- K0) = (and (- r0) (- rem)))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = (and (+ rsel) (and (+ RES) (- rsel))))
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [14]:
(define/ppl case1-step7
  replace*
  case1-step6
  [(and (or (+ r0) (+ rem)) (+ r0)) (+ r0)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (+ r0)))
((- K0) = (and (- r0) (- rem)))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = (and (+ rsel) (and (+ RES) (- rsel))))
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


This next step can be taken because `(and (+ rsel) (- rsel)` must be false.

In [15]:
(define/ppl case1-step8
  replace*
  case1-step7
  [(and (+ rsel) (and (+ RES) (- rsel))) false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (+ r0)))
((- K0) = (and (- r0) (- rem)))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = false)
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [16]:
(define/ppl case1-step9
  propigate/remove*
  case1-step8
  (+ rem) (- rem) (+ lem) (- lem))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (and (+ rsel) (+ RES)) (+ r0)))
((- K0) = (and (- r0) (or (- rsel) (or (- RES) (+ rsel)))))


Just some rearangings

In [17]:
(define/ppl case1-step10
  replace* case1-step9
  [(and (and (+ rsel) (+ RES)) (+ r0))
   (and (+ r0) (and (+ rsel) (+ RES)))]
  [(and (- r0) (or (- rsel) (or (- RES) (+ rsel))))
   (and (- r0) (or (- RES) (or (- rsel) (+ rsel))))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (and (+ rsel) (+ RES))))
((- K0) = (and (- r0) (or (- RES) (or (- rsel) (+ rsel)))))


# CASE 2 `+GO` = 1 and `-GO` = 0 (`GO` is 1)

In [18]:
(define/ppl case2-step1 #:no-check
  (replace* step-before-case
            [(+ GO) true]
            [(- GO) false]))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or true (+ lem)))
((- left) = (and false (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or true (+ r0)))
((- both) = (and false (- r0)))


In [19]:
(define/ppl case2-step2
             replace* case2-step1
             [(or true (+ lem)) true]
             [(and false (- lem)) false]
             [(or true (+ r0)) true]
             [(and false (- r0)) false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = true)
((- left) = false)
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = true)
((- both) = false)


In [20]:
(define/ppl case2-step3
             propigate/remove* case2-step2
                               (+ left) (- left)
                               (+ both) (- both))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and true (and (+ right) true)))
((- K0) = (or false (or (- right) false)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [21]:
(define/ppl case2-step4
             replace* case2-step3
             [(and true (and (+ right) true))
              (+ right)]
             [(or false (or (- right) false))
              (- right)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (+ right))
((- K0) = (- right))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [22]:
(define/ppl case2-step5 
             propigate/remove* case2-step4
             (+ right) (- right)
             (+ rem) (- rem)
             (+ lem) (- lem))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (or (+ r0) (and (+ SEL) (and (+ RES) (- rsel)))))
((- K0) = (and (- r0) (or (- SEL) (or (- RES) (+ rsel)))))


In [23]:
(define/ppl case2-step6
             replace* case2-step5
                      [(and (+ r0) (+ r0)) (+ r0)]
                      [(and (- r0) true) (- r0)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (or (+ r0) (and (+ SEL) (and (+ RES) (- rsel)))))
((- K0) = (and (- r0) (or (- SEL) (or (- RES) (+ rsel)))))


In [24]:
(define/ppl case2-step7
            propigate*
            case2-step6
            (+ SEL) (- SEL))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (or (+ r0) (and (+ rsel) (and (+ RES) (- rsel)))))
((- K0) = (and (- r0) (or (- rsel) (or (- RES) (+ rsel)))))


The next step can be taken because `(and (+ rsel) (- rsel))` muct be false.

In [25]:
(define/ppl case2-step8
  replace*
  case2-step7
  [(and (+ rsel) (and (+ RES) (- rsel))) false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (or (+ r0) false))
((- K0) = (and (- r0) (or (- rsel) (or (- RES) (+ rsel)))))


In [26]:
(define/ppl case2-step9
  replace*
  case2-step8
  [(or (+ r0) false) (+ r0)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (+ r0))
((- K0) = (and (- r0) (or (- rsel) (or (- RES) (+ rsel)))))


some re-aranging

In [27]:
(define/ppl case2-step10
  replace*
  case2-step9
  [(and (- r0) (or (- rsel) (or (- RES) (+ rsel))))
   (and (- r0) (or (- RES) (or (- rsel) (+ rsel))))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (+ r0))
((- K0) = (and (- r0) (or (- RES) (or (- rsel) (+ rsel)))))


# CASE 3 `+GO` = 0 and `-GO` = 1 (`GO` is 0)

In [28]:
(define/ppl case3-step1 #:no-check
   (replace* step-before-case
            [(+ GO) false]
            [(- GO) true]))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (or false (+ lem)))
((- left) = (and true (- lem)))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (or false (+ r0)))
((- both) = (and true (- r0)))


In [29]:
(define/ppl case3-step2
  replace* case3-step1
  [(or false (+ lem)) (+ lem)]
  [(and true (- lem)) (- lem)]
  [(or false (+ r0)) (+ r0)]
  [(and true (- r0)) (- r0)])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ left) (and (+ right) (+ both))))
((- K0) = (or (- left) (or (- right) (- both))))
((+ left) = (+ lem))
((- left) = (- lem))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))
((+ both) = (+ r0))
((- both) = (- r0))


In [30]:
(define/ppl case3-step3
  propigate/remove* case3-step2
  (+ left) (- left)
  (+ both) (- both))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (+ right) (+ r0))))
((- K0) = (or (- lem) (or (- right) (- r0))))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [31]:
(define/ppl case3-step4
  propigate*
  case3-step3
  (+ SEL) (- SEL))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (+ right) (+ r0))))
((- K0) = (or (- lem) (or (- right) (- r0))))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = (and (+ rsel) (and (+ RES) (- rsel))))
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


again, because `(and (+ rsel) (- rsel))`

In [32]:
(define/ppl case3-step5
  replace* case3-step4
  [(and (+ rsel) (and (+ RES) (- rsel))) 
   false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ lem) (and (+ right) (+ r0))))
((- K0) = (or (- lem) (or (- right) (- r0))))
((+ right) = (or (+ r0) (+ rem)))
((- right) = (and (- r0) (- rem)))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = false)
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [33]:
(define/ppl case3-step6
 propigate/remove* case3-step5
 (+ rem) (- rem)
 (+ lem) (- lem)
 (+ right) (- right))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (and (+ rsel) (+ RES)) (and (or (+ r0) false) (+ r0))))
((- K0)
 =
 (or (or (- rsel) (- RES))
     (or (and (- r0) (or (- rsel) (or (- RES) (+ rsel)))) (- r0))))


In [34]:
(define/ppl case3-step7
 replace* case3-step6
 [(and (or (+ r0) false) (+ r0))
  (+ r0)]
 [(or (and (- r0) (or (- rsel) (or (- RES) (+ rsel)))) (- r0))
  (- r0)])
  

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (and (+ rsel) (+ RES)) (+ r0)))
((- K0) = (or (or (- rsel) (- RES)) (- r0)))


## QED

In [35]:
(define/ppl simp1
  propigate/remove*
  step-before-case
  (+ both) (- both) 
  (+ left) (- left)
  (+ right) (- right))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0)
 =
 (and (or (+ GO) (+ lem)) (and (or (+ r0) (+ rem)) (or (+ GO) (+ r0)))))
((- K0)
 =
 (or (and (- GO) (- lem)) (or (and (- r0) (- rem)) (and (- GO) (- r0)))))
((+ lem) = (and (+ SEL) (+ RES)))
((- lem) = (or (- SEL) (- RES)))
((+ rem) = (and (+ SEL) (and (+ RES) (- rsel))))
((- rem) = (or (- SEL) (or (- RES) (+ rsel))))


In [36]:
(define/ppl simp2
  propigate*
  simp1
  (+ SEL) (- SEL))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0)
 =
 (and (or (+ GO) (+ lem)) (and (or (+ r0) (+ rem)) (or (+ GO) (+ r0)))))
((- K0)
 =
 (or (and (- GO) (- lem)) (or (and (- r0) (- rem)) (and (- GO) (- r0)))))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = (and (+ rsel) (and (+ RES) (- rsel))))
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [37]:
(define/ppl simp3
  replace*
  simp2
  [(and (or (+ r0) (+ rem)) (or (+ GO) (+ r0)))
   (or (+ r0) (and (+ rem) (+ GO)))]
  [(or (and (- r0) (- rem)) (and (- GO) (- r0)))
   (and (- r0) (or (- rem) (- GO)))]
  [(and (+ rsel) (and (+ RES) (- rsel)))
   false])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (or (+ GO) (+ lem)) (or (+ r0) (and (+ rem) (+ GO)))))
((- K0) = (or (and (- GO) (- lem)) (and (- r0) (or (- rem) (- GO)))))
((+ lem) = (and (+ rsel) (+ RES)))
((- lem) = (or (- rsel) (- RES)))
((+ rem) = false)
((- rem) = (or (- rsel) (or (- RES) (+ rsel))))


In [38]:
(define/ppl simp4
  propigate/remove*
  simp3
  (+ lem) (- lem)
  (+ rem) (- rem))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0)
 =
 (and (or (+ GO) (and (+ rsel) (+ RES))) (or (+ r0) (and false (+ GO)))))
((- K0)
 =
 (or (and (- GO) (or (- rsel) (- RES)))
     (and (- r0) (or (or (- rsel) (or (- RES) (+ rsel))) (- GO)))))


In [39]:
(define/ppl simp5
  replace* simp4
  [(or (+ r0) (and false (+ GO))) (+ r0)]
  [(or (or (- rsel) (or (- RES) (+ rsel))) (- GO))
   (or (or (- rsel) (- RES)) (or (+ rsel) (- GO)))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (or (+ GO) (and (+ rsel) (+ RES))) (+ r0)))
((- K0)
 =
 (or (and (- GO) (or (- rsel) (- RES)))
     (and (- r0) (or (or (- rsel) (- RES)) (or (+ rsel) (- GO))))))


In [40]:
(define/ppl simp6
  replace* simp5
  [(and (or (+ GO) (and (+ rsel) (+ RES))) (+ r0))
   (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES))))])
   

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES)))))
((- K0)
 =
 (or (and (- GO) (or (- rsel) (- RES)))
     (and (- r0) (or (or (- rsel) (- RES)) (or (+ rsel) (- GO))))))


In [41]:
(define/ppl simp7
  replace* simp6
  [(or (and (- GO) (or (- rsel) (- RES)))
       (and (- r0) (or (or (- rsel) (- RES)) (or (+ rsel) (- GO)))))
   ;; CNF
   (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
        (and (or (- GO) (- r0)) 
             (or (- rsel) (or (- RES) (- r0)))))])

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES)))))
((- K0)
 =
 (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
      (and (or (- GO) (- r0)) (or (- rsel) (or (- RES) (- r0))))))


In [42]:
(define/ppl simp8
  replace* simp7
  [(and (or (- GO) (- r0)) 
        (or (- rsel) (or (- RES) (- r0))))
   (or (- r0)
       (and (- GO) (or (- rsel) (- RES))))])
   

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ K0) = (and (+ r0) (or (+ GO) (and (+ rsel) (+ RES)))))
((- K0)
 =
 (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
      (or (- r0) (and (- GO) (or (- rsel) (- RES))))))


In [43]:
(define/ppl expanded #:no-check
 (((+ SEL) = (+ rsel))
  ((- SEL) = (- rsel))
  ((+ is-res) = (and (+ rsel) (+ RES)))
  ((- is-res) = (or (- rsel) (- RES)))
  ((+ active) = (or (+ GO) (+ is-res)))
  ((- active) = (and (- GO) (- is-res)))
  ((+ K0) = (and (+ r0) (+ active)))
  ((- K0)
   =
   (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel))))
        (or (- r0) (- active))))))

((+ SEL) = (+ rsel))
((- SEL) = (- rsel))
((+ is-res) = (and (+ rsel) (+ RES)))
((- is-res) = (or (- rsel) (- RES)))
((+ active) = (or (+ GO) (+ is-res)))
((- active) = (and (- GO) (- is-res)))
((+ K0) = (and (+ r0) (+ active)))
((- K0)
 =
 (and (or (- GO) (or (- RES) (or (- rsel) (+ rsel)))) (or (- r0) (- active))))


Hypothesis: `(or (- SEL) (+ SEL))` is true for all circuits, (essientially, SEL is always defined). This gives us:

```
((+ K0) = (and (+ r0) (+ active)))
((- K0) = (or (- r0) (- active))))
 ```

or, in the construtive formulation

```
(K0 = (and r0 active))
```.

Theorem: `K0 => active` for all circuits: that is unless something is being resumed or initially activatd it cannot K0. This is follows from Posulate 1. 

Thus, if Hypothesis 1 holds, then K0 = R0, and we are done.